### Data Analysis

In [4]:
import requests
import pandas as pd

url = "https://file.notion.so/f/f/f3d54618-4ee1-480d-b8f7-25c361e2c7b2/12d14740-f5b4-4ce9-9d4d-b9d0b3cd5548/data.parquet?table=block&id=6fa5d867-4063-4844-aebb-4be60a25af39&spaceId=f3d54618-4ee1-480d-b8f7-25c361e2c7b2&expirationTimestamp=1764360000000&signature=wi3c0TpcRhYBT-U1B96Nw6-m3e2vU_wKHRpw45ZUmpA&downloadName=data.parquet"

r = requests.get(url)
with open("data.parquet", "wb") as f:
    f.write(r.content)

df = pd.read_parquet("data.parquet")
print(df)

                     banknifty     nifty  tte
time                                         
2021-01-01 09:15:00   0.286058  0.199729   27
2021-01-01 09:16:00   0.285381  0.200433   27
2021-01-01 09:17:00   0.284233  0.200004   27
2021-01-01 09:18:00   0.286104  0.199860   27
2021-01-01 09:19:00   0.285539  0.198951   27
...                        ...       ...  ...
2022-06-30 15:26:00   0.240701  0.214758   28
2022-06-30 15:27:00   0.240875  0.216558   28
2022-06-30 15:28:00   0.242115  0.216794   28
2022-06-30 15:29:00   0.243426  0.216455   28
2022-06-30 15:30:00   0.241907  0.216081   28

[690512 rows x 3 columns]


In [9]:
df.isna()
df.isna().sum()

banknifty    1500
nifty        2000
tte             0
dtype: int64

In [15]:
####### I have to find, how many times Consecutive missing blocks are there

### 1.I can remove this row completely
### 2.Using KNN or mean(upper + lower)/2 to fill the missing value(if not consecutive)
### 3.ffill() or bfill() for consecutive missing values(I can)

# df = df.dropna(how='any')
# df.isna().sum()

df = df.ffill()
df.isna().sum()


banknifty    0
nifty        0
tte          0
dtype: int64